In [1]:
import argparse
import json
import sys
import time

import zmq
from auxiliaries.detector import Detector
from auxiliaries.utility import *
from models.deeplog import deeplog

# sys.path.append('PWD')
# print(sys.path)

In [2]:
# Config Parameters

options = dict()
# options["data_dir"] = "../data/"
# options["window_size"] = 10
options["device"] = "cpu"

# Smaple
# options["sample"] = "sliding_window"
# options["window_size"] = 10  # if fix_window

# Features
# options["sequentials"] = True
# options["quantitatives"] = False
# options["semantics"] = False
# options["feature_num"] = sum(
#    [options["sequentials"], options["quantitatives"], options["semantics"]]
# )

# Model
options["input_size"] = 1
options["hidden_size"] = 64
options["num_layers"] = 2
options["num_classes"] = 269

# Train
# options["batch_size"] = 2048
# options["accumulation_step"] = 1

# options["optimizer"] = "adam"
# options["lr"] = 0.001
# options["max_epoch"] = 370
# options["lr_step"] = (300, 350)
# options["lr_decay_ratio"] = 0.1

# options["resume_path"] = None
# options["model_name"] = "audit"
# options["save_dir"] = "/result/deeplog/"

# Predict
options[
    "model_path"
] = "/home/aca/Documents/AD_AIT_ACA_Imp/result/deeplog/audit_last.pth"
options["num_candidates"] = 9

seed_everything(seed=1234)


def train():
    Model = deeplog(
        input_size=options["input_size"],
        hidden_size=options["hidden_size"],
        num_layers=options["num_layers"],
        num_keys=options["num_classes"],
    )
    trainer = Trainer(Model, options)
    trainer.start_train()

In [3]:
# load the model
Model = deeplog(
    input_size=options["input_size"],
    hidden_size=options["hidden_size"],
    num_layers=options["num_layers"],
    num_keys=options["num_classes"],
)
dectector = Detector(Model, options)
loaded_model = dectector.load_model()

# establish the zmq connection

context = zmq.Context()

# creating Two soket one for sending and one for eciving
zmq_pub_socket = context.socket(zmq.PUB)
zmq_pub_socket.connect("tcp://127.0.0.1:5560")

zmq_sub_socket = context.socket(zmq.SUB)
zmq_sub_socket.connect("tcp://127.0.0.1:5559")
zmq_sub_socket.setsockopt_string(zmq.SUBSCRIBE, "AminerOutput")

model_path: /home/aca/Documents/AD_AIT_ACA_Imp/result/deeplog/audit_last.pth


In [4]:
# Start your result manager and workers before you start your producers
while True:
    time.sleep(2)
    msg = zmq_sub_socket.recv()
    top, seq = msg.split(b":")
    seq = json.loads(seq)
    seq = [int(x) for x in seq]
    result = dectector.detect_anomaly(loaded_model, seq[:-1], seq[-1])
    output = [seq[:-1], seq[-1], result]
    print(output)
    time.sleep(2)
    zmq_pub_socket.send_string("DetectorOutput:" + json.dumps(output))

[[0, 2, 0, 0, 2, 2, 2, 0, 2, 2], 2, False]
[[2, 0, 0, 2, 2, 2, 0, 2, 2, 2], 42, False]
[[0, 0, 2, 2, 2, 0, 2, 2, 2, 42], 2, False]
[[0, 2, 2, 2, 0, 2, 2, 2, 42, 2], 0, False]
[[2, 2, 2, 0, 2, 2, 2, 42, 2, 0], 0, False]
[[2, 2, 0, 2, 2, 2, 42, 2, 0, 0], 0, False]
[[2, 0, 2, 2, 2, 42, 2, 0, 0, 0], 2, False]
[[0, 2, 2, 2, 42, 2, 0, 0, 0, 2], 0, False]
[[2, 2, 2, 42, 2, 0, 0, 0, 2, 0], 0, False]
[[2, 2, 42, 2, 0, 0, 0, 2, 0, 0], 0, False]
[[2, 42, 2, 0, 0, 0, 2, 0, 0, 0], 2, False]
[[42, 2, 0, 0, 0, 2, 0, 0, 0, 2], 0, False]
[[2, 0, 0, 0, 2, 0, 0, 0, 2, 0], 0, False]
[[0, 0, 0, 2, 0, 0, 0, 2, 0, 0], 0, False]
[[0, 0, 2, 0, 0, 0, 2, 0, 0, 0], 2, False]
[[0, 2, 0, 0, 0, 2, 0, 0, 0, 2], 0, False]
[[2, 0, 0, 0, 2, 0, 0, 0, 2, 0], 0, False]
[[0, 0, 0, 2, 0, 0, 0, 2, 0, 0], 0, False]
[[0, 0, 2, 0, 0, 0, 2, 0, 0, 0], 2, False]
[[0, 2, 0, 0, 0, 2, 0, 0, 0, 2], 0, False]
[[2, 0, 0, 0, 2, 0, 0, 0, 2, 0], 0, False]
[[0, 0, 0, 2, 0, 0, 0, 2, 0, 0], 0, False]
[[0, 0, 2, 0, 0, 0, 2, 0, 0, 0], 2, False]


KeyboardInterrupt: 

In [16]:
send_zmq_socket.close()
reciv_zmq_socket.close()